In [1]:
!pip install keras
import pandas as pd
from collections import Counter
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# import torch
# import torch.nn as nn
import matplotlib.pyplot as plt 

Using TensorFlow backend.


In [2]:
df=pd.read_csv('./AmazonDataset/Label70000.csv')
print(len(df))
df = df[df['text'].str.split().str.len().gt(5)]
df = df[df['text'].str.split().str.len().lt(400)]
df=df.reset_index(drop=None)
print(len(df))
# news=df['text']
# label=df['label']
# len(news)
# current_words=[]
# for i in range(len(news)):
#   # for word in news.loc[i].split(" "):
#     current_words.append(news.loc[i])
# print((current_words[0]))

70000
61611


For Bigram Model we will create train and test features


In [3]:
df=df[['text','label']]

In [4]:
df.head()

,text,label
0,nape summit also featur annual chariti luncheo...,1
1,appl huge anticip iphon x samsung comeback tou...,1
2,iphon x best phone money isnt issu appl one ex...,1
3,appl aapl premium devic start cool per month d...,1
4,xs spoton face id facialrecognit technolog fan...,1


In [5]:
df['word_count']=df['text'].apply(lambda x: len(str(x).split(" ")))

In [6]:
df.head()
df.word_count.describe()

count    61611.000000
mean        14.112399
std          8.429047
min          6.000000
25%          9.000000
50%         12.000000
75%         17.000000
max        333.000000
Name: word_count, dtype: float64

Word count ranges from 11 to 2668 in the given news corpus which is huge range

In [7]:
common_words=pd.Series(''.join(df['text']).split()).value_counts()
print(common_words[:10])
print("Not Common words")
print(common_words[-1000:])

appl      40913
inc       15109
aapl      14816
share     12375
nasdaq    10188
rate       7740
report     7593
buy        6617
price      6365
iphon      6316
dtype: int64
Not Common words
linenasdaq         1
bestlook           1
juni               1
het                1
todayloop          1
                  ..
powerhousarticl    1
actionqualcomm     1
aaplvoo            1
beispielsweis      1
cashier            1
Length: 1000, dtype: int64


In [8]:
#Removing the most frequent and least frequrbst words
from nltk.tokenize import word_tokenize
most_common=common_words[:7]
least_common=common_words[-30000:]
texts=[]
labels=[]
def removeWords(words):
    final_words=[]
    for word in words:
        if not (word in most_common or word in least_common):
            final_words.append(word)
    return final_words
for i in range(len(df)):
    if i%100000==0:
        print("------>",i)
    words=word_tokenize(df.loc[i].text)
    # print(df.loc[i].text)
    words=removeWords(words)
    extracted_sentence=' '.join(word for word in words)
    # print(df.loc[i].text)
    texts.append(extracted_sentence)
    labels.append(df.loc[i]['label'])
new_df=pd.DataFrame({
    'text':texts,
    'label':labels
})


------> 0


In [9]:
common_words=pd.Series(''.join(new_df['text']).split()).value_counts()
print(common_words[:10])
print("Not Common words")
print(common_words[-30000:])

buy         6295
price       6257
iphon       6194
compani     6141
stock       5404
hold        5163
research    4683
market      4413
target      3846
new         3524
dtype: int64
Not Common words
saidearn           3
linda              3
purposfive         3
rehlefil           3
hubsaid            3
                  ..
tomacdevic         1
saycould           1
statejanuari       1
citizencampaign    1
trumpreview        1
Length: 30000, dtype: int64


In [10]:
new_df['word_count']=new_df['text'].apply(lambda x: len(str(x).split(" ")))
new_df.word_count.describe()


count    61611.000000
mean        11.822288
std          8.192491
min          1.000000
25%          7.000000
50%         10.000000
75%         14.000000
max        261.000000
Name: word_count, dtype: float64

In [11]:
news=new_df['text']
label=new_df['label']
len(news)
current_words=[]
for i in range(len(news)):
  # for word in news.loc[i].split(" "):
    current_words.append(news.loc[i])
print((current_words[0]))

nape summit also featur annual chariti luncheon tom brokaw keynot speaker new job fair upstream profession educ seminar present industri organ aipn ipaa seg domest intern theater run two day prospect preview capit servic provid twoday expo nearli acr exhibit space


In [12]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download("popular")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
cnt_vectorizer = CountVectorizer(max_df=0.8,min_df=0.02,stop_words=stop_words,
            ngram_range=(2,3))

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/akshaykumar/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/akshaykumar/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/akshaykumar/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/akshaykumar/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/akshaykumar/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/akshaykumar/nltk_data...
[nltk_data]    |   Pa

In [13]:
X_Dataset=cnt_vectorizer.fit_transform(current_words)

In [14]:
list(cnt_vectorizer.vocabulary_.keys())[:10]

['nd quarter',
 'second quarter',
 'set price',
 'price object',
 'research note',
 'target price',
 'price target']

Bi gram model for plot Bar graph



In [15]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf=TfidfTransformer(smooth_idf=True,use_idf=True)
X=tfidf.fit_transform(X_Dataset).toarray()

In [16]:
d=current_words[1]
print(d)
tfidf_vector=tfidf.transform(cnt_vectorizer.transform([d]))
# print(tfidf_vector.toarray()[0][17])

huge anticip iphon x samsung comeback tour follow note debacl big year smartphon industri


In [17]:
X.shape

(61611, 7)

In [35]:
trainX=df['text']
trainY=df['label']
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
pca = PCA(n_components=3)
pComponents= pca.fit_transform(X)
# print((pca.components_ ))
print(pca.explained_variance_ratio_)
X_train, X_test, y_train, y_test = train_test_split(pComponents, trainY, test_size=0.2, random_state=23)


[0.21646509 0.20496981 0.17492736]


In [36]:
print(X_train.shape)
print()

(49288, 3)



In [37]:
print(X_train.shape)
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
clf = LogisticRegression(random_state=10).fit(X_train,y_train)
y_pred=clf.predict(X_test)
acc=accuracy_score(y_test,y_pred)
print(acc)

(49288, 3)
0.6704536233060131


In [34]:
print(X_train.shape)
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import tree
clf = tree.DecisionTreeClassifier(random_state=0).fit(X_train,y_train)
y_pred=clf.predict(X_test)
acc=accuracy_score(y_test,y_pred)
print(acc)

(49288, 3)
0.6602288403797777


In [30]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=1000).fit(X_train,y_train)
y_pred=clf.predict(X_test)
acc=accuracy_score(y_test,y_pred)
print(acc)

0.6602288403797777


In [175]:
# from sklearn.metrics import accuracy_score
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.model_selection import GridSearchCV
# rf = RandomForestClassifier(random_state=43, n_jobs=-1)
# param_grid = { "criterion" : ["gini", "entropy"], "min_samples_leaf" : [1, 5, 10], "min_samples_split" : [2, 4, 10, 12, 16], "n_estimators": [50, 100, 400, 700, 1000]}
# gs = GridSearchCV(estimator=rf, param_grid=param_grid, scoring='accuracy', cv=10, n_jobs=-1)
# gs = gs.fit(X_train, y_train)
# accuracy=gs.best_score_
# print(accuracy)
# print(gs.best_params_)
# print(g_search.best_params_)

In [176]:
len(label)==len(news)

True

In [0]:
# from keras import backend as K
# K.tensorflow_backend._get_available_gpus()

In [50]:
def tokenize(x):
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk
def pad(x, length=None):
    if length is None:
        length = len(max(x, key=len))

    return pad_sequences(x, maxlen=length, padding='post')

In [51]:
tokenize_text=[]
news_label=[]
for n_z in news:
  tokenize_text.append(n_z)
for l in label:
  news_label.append(l)

In [52]:
print(tokenize_text[0])
print(news_label[0])

nape summit also featur annual chariti luncheon tom brokaw keynot speaker new job fair upstream profession educ seminar present industri organ aipn ipaa seg aapl domest intern theater run two day prospect preview capit servic provid twoday expo nearli acr exhibit space
1


In [53]:
news_counter = Counter(set(current_words))
vocab_size=((len(news_counter)))
print(vocab_size)
print((len(set(current_words))))
# print(news_counter.items())

8238
8238


In [54]:
text_tokenized, text_tokenizer=tokenize(tokenize_text)
text_padded=pad(text_tokenized)

In [8]:
print(text_padded.shape)

(12001, 1278)


In [9]:
print((text_padded.shape))
print((text_padded[0]))
trainX=text_padded
# trainX = np.reshape(text_padded, (text_padded.shape[0], 1, 2055))
print(trainX[0].shape)
trainY = pd.get_dummies(news_label).values
print(trainY[0])

(12001, 1278)
[3530 3061   40 ...    0    0    0]
(1278,)
[0 1]


In [11]:
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
t = Tokenizer()
t.fit_on_texts(current_words)
vocab_size = len(t.word_index) + 1
encoded_docs = t.texts_to_sequences(current_words)
max_length = 1278
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')

In [12]:
padded_docs.shape

(12001, 1278)

In [13]:
from numpy import zeros
def load_glove_vector(file):
    f = open(file,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print("words loaded")
    return model

model=load_glove_vector('/content/drive/My Drive/Glove.txt')
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
  # print(word,i)
  embedding_vector = model.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/My Drive/Glove.txt'

In [14]:
from sklearn.model_selection import train_test_split
trainY = pd.get_dummies(label.values).values
print(trainY.shape)
X_train, X_test, y_train, y_test = train_test_split(padded_docs, trainY, test_size=0.2, random_state=40)

(12001, 2)


In [15]:
X_train.shape

(9600, 1278)

In [ ]:
X_train.shape
trainX = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
trainX.shape

In [146]:
# !pip install tensorflow==1.14.0
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Bidirectional
from keras.optimizers import Adam
from keras.layers import Embedding
model = Sequential()
# e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=1500, trainable=False)
# model.add(e)
model.add(Bidirectional(LSTM(128,input_shape=(1,X_train.shape[1]),dropout=0.2)))
model.add(Dense(2,activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.01),metrics=['accuracy'])
model.fit(trainX, y_train, epochs=30, batch_size=32, verbose=1,validation_split=0.2)

ValueError: Please provide as model targets either a single array or a list of arrays. You passed: y=31491    1
33051    1
4236     1
16774    0
21432    1
        ..
11532    1
27640    1
14501    1
30727    0
11590    0
Name: label, Length: 34926, dtype: int64

In [18]:
score, acc = model.evaluate(trainX, y_train)
print('Test score:', score)
print('Test accuracy:', acc)

9600/9600 [==============================] - 0s 45us/step
Test score: 0.6195119785269102
Test accuracy: 0.6959375143051147


In [19]:
testX=np.reshape(X_test, (X_test.shape[0], 1, X_train.shape[1]))
score, acc = model.evaluate(testX, y_test)
print('Test score:', score)
print('Test accuracy:', acc)

2401/2401 [==============================] - 0s 58us/step
Test score: 0.6255119546087321
Test accuracy: 0.6905456185340881
